In [ ]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch
import torch.nn.functional as F

__all__ = ['Res2Net', 'res2net50_v1b', 'res2net101_v1b', 'res2net50_v1b_26w_4s']

model_urls = {
    'res2net50_v1b_26w_4s': 'https://shanghuagao.oss-cn-beijing.aliyuncs.com/res2net/res2net50_v1b_26w_4s-3cf99910.pth',
    'res2net101_v1b_26w_4s': 'https://shanghuagao.oss-cn-beijing.aliyuncs.com/res2net/res2net101_v1b_26w_4s-0812c246.pth',
}


class Bottle2neck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, baseWidth=26, scale=4, stype='normal'):
        """ Constructor
        Args:
            inplanes: input channel dimensionality
            planes: output channel dimensionality
            stride: conv stride. Replaces pooling layer.
            downsample: None when stride = 1
            baseWidth: basic width of conv3x3
            scale: number of scale.
            type: 'normal': normal set. 'stage': first block of a new stage.
        """
        super(Bottle2neck, self).__init__()

        width = int(math.floor(planes * (baseWidth / 64.0)))
        self.conv1 = nn.Conv2d(inplanes, width * scale, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width * scale)

        if scale == 1:
            self.nums = 1
        else:
            self.nums = scale - 1
        if stype == 'stage':
            self.pool = nn.AvgPool2d(kernel_size=3, stride=stride, padding=1)
        convs = []
        bns = []
        for i in range(self.nums):
            convs.append(nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, bias=False))
            bns.append(nn.BatchNorm2d(width))
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList(bns)

        self.conv3 = nn.Conv2d(width * scale, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stype = stype
        self.scale = scale
        self.width = width

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        spx = torch.split(out, self.width, 1)
        for i in range(self.nums):
            if i == 0 or self.stype == 'stage':
                sp = spx[i]
            else:
                sp = sp + spx[i]
            sp = self.convs[i](sp)
            sp = self.relu(self.bns[i](sp))
            if i == 0:
                out = sp
            else:
                out = torch.cat((out, sp), 1)
        if self.scale != 1 and self.stype == 'normal':
            out = torch.cat((out, spx[self.nums]), 1)
        elif self.scale != 1 and self.stype == 'stage':
            out = torch.cat((out, self.pool(spx[self.nums])), 1)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Res2Net(nn.Module):

    def __init__(self, block, layers, baseWidth=26, scale=4, num_classes=1000):
        self.inplanes = 64
        super(Res2Net, self).__init__()
        self.baseWidth = baseWidth
        self.scale = scale
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1, 1, bias=False)
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.AvgPool2d(kernel_size=stride, stride=stride,
                             ceil_mode=True, count_include_pad=False),
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample=downsample,
                            stype='stage', baseWidth=self.baseWidth, scale=self.scale))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, baseWidth=self.baseWidth, scale=self.scale))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def res2net50_v1b(pretrained=False, **kwargs):
    """Constructs a Res2Net-50_v1b lib.
    Res2Net-50 refers to the Res2Net-50_v1b_26w_4s.
    Args:
        pretrained (bool): If True, returns a lib pre-trained on ImageNet
    """
    model = Res2Net(Bottle2neck, [3, 4, 6, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net50_v1b_26w_4s']))
    return model


def res2net101_v1b(pretrained=False, **kwargs):
    """Constructs a Res2Net-50_v1b_26w_4s lib.
    Args:
        pretrained (bool): If True, returns a lib pre-trained on ImageNet
    """
    model = Res2Net(Bottle2neck, [3, 4, 23, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net101_v1b_26w_4s']))
    return model


def res2net50_v1b_26w_4s(pretrained=False, **kwargs):
    """Constructs a Res2Net-50_v1b_26w_4s lib.
    Args:
        pretrained (bool): If True, returns a lib pre-trained on ImageNet
    """
    model = Res2Net(Bottle2neck, [3, 4, 6, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model_state = torch.load('./res2net50_v1b_26w_4s-3cf99910.pth')
        model.load_state_dict(model_state)
        # lib.load_state_dict(model_zoo.load_url(model_urls['res2net50_v1b_26w_4s']))
    return model


def res2net101_v1b_26w_4s(pretrained=False, **kwargs):
    """Constructs a Res2Net-50_v1b_26w_4s lib.
    Args:
        pretrained (bool): If True, returns a lib pre-trained on ImageNet
    """
    model = Res2Net(Bottle2neck, [3, 4, 23, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net101_v1b_26w_4s']))
    return model


def res2net152_v1b_26w_4s(pretrained=False, **kwargs):
    """Constructs a Res2Net-50_v1b_26w_4s lib.
    Args:
        pretrained (bool): If True, returns a lib pre-trained on ImageNet
    """
    model = Res2Net(Bottle2neck, [3, 8, 36, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net152_v1b_26w_4s']))
    return model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False)
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x


class RFB_modified(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(RFB_modified, self).__init__()
        self.relu = nn.ReLU(True)
        self.branch0 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
        )
        self.branch1 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(out_channel, out_channel, kernel_size=(3, 1), padding=(1, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=3, dilation=3)
        )
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 5), padding=(0, 2)),
            BasicConv2d(out_channel, out_channel, kernel_size=(5, 1), padding=(2, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=5, dilation=5)
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(out_channel, out_channel, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=7, dilation=7)
        )
        self.conv_cat = BasicConv2d(4*out_channel, out_channel, 3, padding=1)
        self.conv_res = BasicConv2d(in_channel, out_channel, 1)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x_cat = self.conv_cat(torch.cat((x0, x1, x2, x3), 1))

        x = self.relu(x_cat + self.conv_res(x))
        return x


class aggregation(nn.Module):
    # dense aggregation, it can be replaced by other aggregation previous, such as DSS, amulet, and so on.
    # used after MSF
    def __init__(self, channel):
        super(aggregation, self).__init__()
        self.relu = nn.ReLU(True)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv_upsample1 = BasicConv2d(channel, channel, 3, padding=1)
        self.conv_upsample2 = BasicConv2d(channel, channel, 3, padding=1)
        self.conv_upsample3 = BasicConv2d(channel, channel, 3, padding=1)
        self.conv_upsample4 = BasicConv2d(channel, channel, 3, padding=1)
        self.conv_upsample5 = BasicConv2d(2*channel, 2*channel, 3, padding=1)

        self.conv_concat2 = BasicConv2d(2*channel, 2*channel, 3, padding=1)
        self.conv_concat3 = BasicConv2d(3*channel, 3*channel, 3, padding=1)
        self.conv4 = BasicConv2d(3*channel, 3*channel, 3, padding=1)
        self.conv5 = nn.Conv2d(3*channel, 1, 1)

    def forward(self, x1, x2, x3):
        x1_1 = x1
        x2_1 = self.conv_upsample1(self.upsample(x1)) * x2
        x3_1 = self.conv_upsample2(self.upsample(self.upsample(x1))) \
               * self.conv_upsample3(self.upsample(x2)) * x3

        x2_2 = torch.cat((x2_1, self.conv_upsample4(self.upsample(x1_1))), 1)
        x2_2 = self.conv_concat2(x2_2)

        x3_2 = torch.cat((x3_1, self.conv_upsample5(self.upsample(x2_2))), 1)
        x3_2 = self.conv_concat3(x3_2)

        x = self.conv4(x3_2)
        x = self.conv5(x)

        return x


class PraNet(nn.Module):
    # res2net based encoder decoder
    def __init__(self, channel=32):
        super(PraNet, self).__init__()
        # ---- ResNet Backbone ----
        self.resnet = res2net50_v1b_26w_4s(pretrained=False)
        # ---- Receptive Field Block like module ----
        self.rfb2_1 = RFB_modified(512, channel)
        self.rfb3_1 = RFB_modified(1024, channel)
        self.rfb4_1 = RFB_modified(2048, channel)
        # ---- Partial Decoder ----
        self.agg1 = aggregation(channel)
        # ---- reverse attention branch 4 ----
        self.ra4_conv1 = BasicConv2d(2048, 256, kernel_size=1)
        self.ra4_conv2 = BasicConv2d(256, 256, kernel_size=5, padding=2)
        self.ra4_conv3 = BasicConv2d(256, 256, kernel_size=5, padding=2)
        self.ra4_conv4 = BasicConv2d(256, 256, kernel_size=5, padding=2)
        self.ra4_conv5 = BasicConv2d(256, 1, kernel_size=1)
        # ---- reverse attention branch 3 ----
        self.ra3_conv1 = BasicConv2d(1024, 64, kernel_size=1)
        self.ra3_conv2 = BasicConv2d(64, 64, kernel_size=3, padding=1)
        self.ra3_conv3 = BasicConv2d(64, 64, kernel_size=3, padding=1)
        self.ra3_conv4 = BasicConv2d(64, 1, kernel_size=3, padding=1)
        # ---- reverse attention branch 2 ----
        self.ra2_conv1 = BasicConv2d(512, 64, kernel_size=1)
        self.ra2_conv2 = BasicConv2d(64, 64, kernel_size=3, padding=1)
        self.ra2_conv3 = BasicConv2d(64, 64, kernel_size=3, padding=1)
        self.ra2_conv4 = BasicConv2d(64, 1, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)      # bs, 64, 88, 88
        # ---- low-level features ----
        x1 = self.resnet.layer1(x)      # bs, 256, 88, 88
        x2 = self.resnet.layer2(x1)     # bs, 512, 44, 44

        x3 = self.resnet.layer3(x2)     # bs, 1024, 22, 22
        x4 = self.resnet.layer4(x3)     # bs, 2048, 11, 11
        x2_rfb = self.rfb2_1(x2)        # channel -> 32
        x3_rfb = self.rfb3_1(x3)        # channel -> 32
        x4_rfb = self.rfb4_1(x4)        # channel -> 32

        ra5_feat = self.agg1(x4_rfb, x3_rfb, x2_rfb)
        lateral_map_5 = F.interpolate(ra5_feat, scale_factor=8, mode='bilinear')    # NOTES: Sup-1 (bs, 1, 44, 44) -> (bs, 1, 352, 352)

        # ---- reverse attention branch_4 ----
        crop_4 = F.interpolate(ra5_feat, scale_factor=0.25, mode='bilinear')
        x = -1*(torch.sigmoid(crop_4)) + 1
        x = x.expand(-1, 2048, -1, -1).mul(x4)
        x = self.ra4_conv1(x)
        x = F.relu(self.ra4_conv2(x))
        x = F.relu(self.ra4_conv3(x))
        x = F.relu(self.ra4_conv4(x))
        ra4_feat = self.ra4_conv5(x)
        x = ra4_feat + crop_4
        lateral_map_4 = F.interpolate(x, scale_factor=32, mode='bilinear')  # NOTES: Sup-2 (bs, 1, 11, 11) -> (bs, 1, 352, 352)

        # ---- reverse attention branch_3 ----
        crop_3 = F.interpolate(x, scale_factor=2, mode='bilinear')
        x = -1*(torch.sigmoid(crop_3)) + 1
        x = x.expand(-1, 1024, -1, -1).mul(x3)
        x = self.ra3_conv1(x)
        x = F.relu(self.ra3_conv2(x))
        x = F.relu(self.ra3_conv3(x))
        ra3_feat = self.ra3_conv4(x)
        x = ra3_feat + crop_3
        lateral_map_3 = F.interpolate(x, scale_factor=16, mode='bilinear')  # NOTES: Sup-3 (bs, 1, 22, 22) -> (bs, 1, 352, 352)

        # ---- reverse attention branch_2 ----
        crop_2 = F.interpolate(x, scale_factor=2, mode='bilinear')
        x = -1*(torch.sigmoid(crop_2)) + 1
        x = x.expand(-1, 512, -1, -1).mul(x2)
        x = self.ra2_conv1(x)
        x = F.relu(self.ra2_conv2(x))
        x = F.relu(self.ra2_conv3(x))
        ra2_feat = self.ra2_conv4(x)
        x = ra2_feat + crop_2
        lateral_map_2 = F.interpolate(x, scale_factor=8, mode='bilinear')   # NOTES: Sup-4 (bs, 1, 44, 44) -> (bs, 1, 352, 352)

        return lateral_map_5, lateral_map_4, lateral_map_3, lateral_map_2

In [ ]:
DIV_COEFFICIENT = 4
DEVICE='cpu'

In [ ]:
net = PraNet()
net.load_state_dict(torch.load("../input/last-best-model/best_model_weights_0.4900_0.9474.save", map_location=DEVICE))
threshold = 0.49
net = net.float().eval()
net.to(device=DEVICE);

In [ ]:
import os
import tifffile
import matplotlib.pyplot as plt

TEST_PATH = "../input/hubmap-kidney-segmentation/test"

#import json

In [ ]:
import numpy as np


def handle_by_crops(images, network, dsr_size, margin_size, device=DEVICE):
    #print(images.shape)
    height = images.shape[-2]  # numpy has opposite dimension order
    width = images.shape[-1]
    lines = []
    network.to(device=device)
    x_prev = 0
    sigmoid = nn.Sigmoid()
    for x in range(dsr_size, height+dsr_size - 2 * margin_size - 1, dsr_size - 2 * margin_size):
        x = min(x, height)
        line = []
        y_prev = 0
        for y in range(dsr_size, width+dsr_size - 2 * margin_size - 1, dsr_size - 2*margin_size):
            y = min(y, width)
            img_cropped = images[:,:,x-dsr_size:x, y - dsr_size: y]
            img_cropped = img_cropped.to(device=device)
            
            
            pred = network(img_cropped)
            # the sum is the best to use (see notebook best_way_to_threshold)
            mask = sigmoid(pred[0] + pred[1] + pred[2] + pred[3])  
            mask = mask[:,0].detach().cpu().numpy()
            if y - dsr_size != 0:
                mask = mask[:,:,margin_size:]

            if x - dsr_size!= 0:
                mask = mask[:,margin_size:]

            if y != width:
                mask = mask[:,:,:-margin_size]
            else:
                mask = mask[:,:,y_prev-(y-dsr_size + margin_size):]


            if x != height:
                mask = mask[:,:-margin_size]
            else:
                mask = mask[:,x_prev-(x-dsr_size + margin_size):]
            
            #print(mask.shape)
            #print()
            mask = mask * 255
            mask = mask.astype('uint8')
            line.append(mask)
            y_prev = y - margin_size


        lines.append(np.concatenate(line,axis=2))
        x_prev =  x - margin_size
        
    result_mask = np.concatenate(lines, axis=1)
    #print(result_mask.shape)
    return result_mask


In [ ]:
# def mask2rle(pixels):
#     '''
#     img: numpy array, 1 - mask, 0 - background
#     Returns run length as string formated
#     '''
    
#     pixels = np.concatenate([[0], pixels, [0]])
#     runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
#     runs[1::2] -= runs[::2]
#     return ' '.join(str(x) for x in runs)

In [ ]:
images = [x[:-5] for x in os.listdir(TEST_PATH) if 'tiff' in x]

In [ ]:
threshold = int(255 * threshold)
threshold

In [ ]:
%%time
from PIL import Image
import cv2
import gc
import pickle
Image.MAX_IMAGE_PIXELS = 1731207120


#net.eval()

imgs = []
masks = []
for i,image in enumerate(images):
    print(image)
    tiff_file_path = os.path.join(TEST_PATH, image + '.tiff')
    anatomical_structure_file_path = os.path.join(TEST_PATH, image + '-anatomical-structure.json')
    
    # Open the file
    original_image = tifffile.imread(tiff_file_path)
    if len(original_image.shape) == 5:
        original_image = original_image[0][0].transpose(1,2,0)

    if original_image.shape[0] == 3: # Channels are first
        original_image = original_image.transpose(1,2,0)
    
    print(original_image.shape)
    original_height = original_image.shape[0]
    original_width = original_image.shape[1]
    height = original_height // DIV_COEFFICIENT  # numpy has height-then-width orientation
    width = original_width // DIV_COEFFICIENT
    original_image = np.array(Image.fromarray(original_image).resize((width, height)))
    gc.collect()
    original_image = original_image.transpose(2,0,1) / 255
    original_image_torch = torch.from_numpy(original_image)[None].type(torch.FloatTensor)
    
    del original_image
    gc.collect()
    
    mask_predicted = handle_by_crops(original_image_torch, net, 224*3, 112)
    del original_image_torch
    gc.collect()
    mask_predicted = cv2.resize(mask_predicted[0], dsize=(original_width, original_height), interpolation=cv2.INTER_CUBIC)
    #mask_predicted = Image.fromarray(mask_predicted[0])
    #mask_predicted = mask_predicted.resize((original_width, original_height))
    
    gc.collect()
    print(mask_predicted.shape)
    mask_predicted = mask_predicted > threshold   # IT WILL BREAKS BECAUSE OF MEMORY HERE, THOUGH RIGHT BEFORE IT IS ONLY CONSUMED 3GB
    gc.collect()
    
    #mask_predicted = np.array(mask_predicted)
    #mask_predicted = np.array(Image.fromarray(mask_predicted[0]).resize((original_width, original_height)))
    print("create mask")
    
    mask_predicted = mask_predicted.T
    gc.collect()
    mask_predicted = mask_predicted.flatten()
    gc.collect()
    mask_predicted = np.concatenate([[0], mask_predicted, [0]])
    print("concated")
    mask_predicted = np.where(mask_predicted[1:] != mask_predicted[:-1])[0] + 1
    print("found")
    mask_predicted[1::2] -= mask_predicted[::2]
    print("almost")
    rle_mask = ' '.join(str(x) for x in mask_predicted)
    #return ' '.join(str(x) for x in runs)
    print("done")
    imgs.append(image)
    masks.append(rle_mask) 
    del mask_predicted, rle_mask
    gc.collect()

In [ ]:
#threshold

In [ ]:
#import matplotlib.pyplot as plt

In [ ]:
#len(rle_mask)

In [ ]:
#mask_predicted.shape

In [ ]:

#runs = np.where(mask_predicted[1:] != mask_predicted[:-1])[0] + 1
#runs[1::2] -= runs[::2]
#return ' '.join(str(x) for x in runs)

In [ ]:
import pandas as pd
answ_df = pd.DataFrame()
answ_df['id'] = imgs
answ_df['predicted'] = masks
answ_df.to_csv("./submission.csv")
answ_df

In [ ]:
# import json

# def get_coordinates_from_structure_file(structure_file_path, coef=DIV_COEFFICIENT):
#     with open(structure_file_path, "r") as file:
#         anatomical_structure_file = json.load(file)
    
#     lists_of_coordinates = []
#     for structure in anatomical_structure_file:
#         if 'properties' in structure:
#             if 'classification' in structure['properties']:
#                 if 'name' in structure['properties']['classification']:
#                     name = structure['properties']['classification']['name']
#                     if name == 'Cortex':
#                         if 'geometry' in structure:
#                             if 'coordinates' in structure['geometry']:
#                                 coordinates = structure['geometry']['coordinates']
#                                 if len(coordinates) == 1:
#                                     coordinates = coordinates[0]

#                                 x = [x[0]/coef for x in coordinates]
#                                 y = [x[1]/coef for x in coordinates]
#                                 coordinates = list(zip(x,y))
#                                 lists_of_coordinates.append(coordinates)
    
#     return lists_of_coordinates

In [ ]:
# %%time
# from PIL import Image
# import gc
# import pickle



# def get_mask(mask_predicted, seg_map):
#     return ((mask_predicted > threshold)*seg_map).astype('uint8')


# Image.MAX_IMAGE_PIXELS = 1731207120

# DIV_COEFFICIENT = 4

# net.eval()

# imgs = []
# masks_masked_by_annotation = []
# masks_no_masked = []
# for i,image in enumerate(images):
#     print(image)
    
#     tiff_file_path = os.path.join(TEST_PATH, image + '.tiff')
#     anatomical_structure_file_path = os.path.join(TEST_PATH, image + '-anatomical-structure.json')
#     coordinates = get_coordinates_from_structure_file(anatomical_structure_file_path, 1)
    
#     # Open the file
#     original_image = tifffile.imread(tiff_file_path)
#     if len(original_image.shape) == 5:
#         original_image = original_image[0][0].transpose(1,2,0)
        
#     if original_image.shape[0] == 3: # Channels are first
#         original_image = original_image.transpose(1,2,0)
    
#     print(original_image.shape)
#     original_height = original_image.shape[0]
#     original_width = original_image.shape[1]
#     height = original_height // DIV_COEFFICIENT  # numpy has height-then-width orientation
#     width = original_width // DIV_COEFFICIENT
#     original_image = np.array(Image.fromarray(original_image).resize((width, height)))
#     original_image = original_image.transpose(2,0,1) / 255
#     original_image_torch = torch.from_numpy(original_image)[None].type(torch.FloatTensor)
#     original_image  = None
#     gc.collect()
#     print("hi")
#     mask_predicted = handle_by_crops(original_image_torch, net, 224*3, 112)
#     gc.collect()
#     print("buy")
#     mask_predicted = mask_predicted[0]
#     mask_predicted = np.array(Image.fromarray(mask_predicted).resize((original_width, original_height)))
#     seg_map = make_seg_map(mask_predicted.shape, coordinates)
#     mask_masked_by_annotation = get_mask(mask_predicted > threshold,seg_map)
#     mask_no_masked = get_mask(mask_predicted > threshold,1.0)
    
#     imgs.append(image)
#     masks_masked_by_annotation.append(mask2rle(mask_masked_by_annotation))
#     masks_no_masked.append(mask2rle(mask_no_masked))
#     #break
#     gc.collect()

In [ ]:
# answ_df = pd.DataFrame()
# answ_df['id'] = imgs
# answ_df['predicted'] = masks_masked_by_annotation
# answ_df.to_csv("./submission.csv")
# answ_df